Інсталюємо необхідні бібліотеки, такі як `transformers` для роботи з моделями, `peft` для тонкого налаштування, `accelerate` для оптимізації обчислень, і `datasets` для завантаження та обробки датасетів.

In [1]:
%pip install transformers peft accelerate datasets
%pip install --upgrade torch


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ["HF_TOKEN"] = "<my_token>"

In [3]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

**Імпорти і підготовка моделі**:
   - Імпортуються `AutoModelForCausalLM` та `AutoTokenizer` із `transformers`.
   - Створюється токенізатор і модель `Qwen2.5-0.5B-Instruct`, які завантажуються з бібліотеки Hugging Face і налаштовуються для обчислень на доступному пристрої (наприклад, `MPS` або CPU).


 **Тестова генерація тексту**:
   - Модель генерує відповідь на підготовлений текстовий запит (наприклад, створення вірша). Генерація відбувається за допомогою функції `generate`, після чого текст декодується у зрозумілий формат.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # Use float32 for MPS
    device_map={"": device},
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = model.to(device)

print("Model moved to", device)

prompt = "Напиши красивий вірш"
messages = [
    {
        "role": "system",
        "content": "SerGPT Zhadan - найкращий поет української сучасності",
    },
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

/Users/yuri/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model moved to mps
"Завжди, я жодне та радостно, 
Мені це з'єднюються все.
Користуються ми, якісь,
Світлюючи нашу любовь.

Вірш з певним змогом,
Ось наші думки, будь ласка!
Добре відрізнятися нашими думками,
Цей вірш - наш розвиток. 

Звичайно, це вірш,
Уважливий і вірний!
Безпека і спокійна душа,
Тому вірш з цим назвою зберігаються.

Завжди, я жодне та радостно,
Мені це з'єднюють все."


**Завантаження датасету**:
   - Завантажується датасет з файлу `poems/poems-qwen.json` для тренування і валідації моделі.
   - Дані розділяються на навчальну та тестову вибірки.


In [5]:
from datasets import load_dataset

# Завантаження датасету
dataset_path = "poems/poems-qwen.json"
dataset = load_dataset('json', data_files=dataset_path)

# Розділіть дані на навчальні та валідаційні
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


**Токенізація датасету**:
   - Дані токенізуються з використанням токенізатора моделі. Об'єднуються поля "інструкція" і "вхідні дані", після чого виконуються операції токенізації із вказаними обмеженнями на довжину.

In [6]:
def tokenize_function(examples):
    # Combine each instruction and input pair into a single string
    combined_inputs = [inst + inp for inst, inp in zip(examples["instruction"], examples["input"])]
    
    return tokenizer(
        combined_inputs,
        text_target=examples["output"],
        padding="max_length",
        max_length=512,
        truncation=True
    )


tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 30/30 [00:00<00:00, 2002.28 examples/s]


**Налаштування тренування**:
   - Використовується клас `Trainer` з `transformers`. Налаштовуються параметри тренування, такі як кількість епох, розмір пакету, швидкість навчання, і використання CPU або GPU.


In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=4,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False,
    # Remove use_mps_device
    fp16=False,
    bf16=False,
    use_cpu=True,  # Force CPU and then manually move model to MPS if you like
)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model.gradient_checkpointing_enable()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
)

**Тренування моделі**:
   - Починається процес тренування з логуванням прогресу і результатів після кожної епохи, включаючи втрати (`loss`) і результати оцінки (`eval_loss`).

In [8]:
trainer.train()


 10%|█         | 10/99 [04:03<35:28, 23.92s/it]

{'loss': 35.664, 'grad_norm': 472.41082763671875, 'learning_rate': 4.494949494949495e-05, 'epoch': 0.3}


 20%|██        | 20/99 [08:01<31:10, 23.67s/it]

{'loss': 21.4517, 'grad_norm': 516.3314819335938, 'learning_rate': 3.98989898989899e-05, 'epoch': 0.61}


 30%|███       | 30/99 [11:44<26:03, 22.65s/it]

...

 91%|█████████ | 90/99 [37:47<03:36, 24.01s/it]

{'loss': 20.8111, 'grad_norm': 187.64077758789062, 'learning_rate': 4.5454545454545455e-06, 'epoch': 2.73}


                                               
100%|██████████| 99/99 [42:08<00:00, 25.54s/it]

{'eval_loss': 4.466160297393799, 'eval_runtime': 23.9766, 'eval_samples_per_second': 1.251, 'eval_steps_per_second': 0.167, 'epoch': 3.0}
{'train_runtime': 2528.2027, 'train_samples_per_second': 0.312, 'train_steps_per_second': 0.039, 'train_loss': 21.795295330009075, 'epoch': 3.0}


TrainOutput(global_step=99, training_loss=21.795295330009075, metrics={'train_runtime': 2528.2027, 'train_samples_per_second': 0.312, 'train_steps_per_second': 0.039, 'total_flos': 867476307050496.0, 'train_loss': 21.795295330009075, 'epoch': 3.0})

**Збереження моделі**:
   - Після завершення тренування модель і токенізатор зберігаються в директорію `qwen-fine_tuned_model`.


In [9]:
model.save_pretrained("qwen-fine_tuned_model")
tokenizer.save_pretrained("qwen-fine_tuned_model")

('qwen-fine_tuned_model/tokenizer_config.json',
 'qwen-fine_tuned_model/special_tokens_map.json',
 'qwen-fine_tuned_model/vocab.json',
 'qwen-fine_tuned_model/merges.txt',
 'qwen-fine_tuned_model/added_tokens.json',
 'qwen-fine_tuned_model/tokenizer.json')

**Фінальне тестування**:
   - Виконується генерація тексту на основі навченої моделі для перевірки її роботи.


In [10]:
prompt = "Напиши красивий вірш"
messages = [
    {
        "role": "system",
        "content": "SerGPT Zhadan - найкращий поет української сучасності",
    },
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

іі,

юх,в

 в,іія т піє,

ио � зд

і


